In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from skimage.feature import local_binary_pattern, hog
from skimage.color import rgb2gray
from sklearn.preprocessing import StandardScaler
from skimage.transform import resize
from sklearn.svm import SVR
from skimage import exposure
from scipy.stats import spearmanr
from skimage import color

In [2]:
class FacesMemorabilityDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[index, 0])
        image = Image.open(img_name).convert('RGB')
        memorability_score = self.dataframe.iloc[index, 6] - self.dataframe.iloc[index, 7]
        if self.transform:
            image = self.transform(image)
        return image, memorability_score

In [3]:
def extract_color_features(image):
    # 将PIL.Image对象转换为numpy数组
    image_array = np.array(image)
    # 计算彩色直方图
    color_features = [np.histogram(image_array[:, :, i], bins=256, range=(0, 256))[0] for i in range(3)]
    return np.concatenate(color_features)

def extract_lbp_features(image):
    gray_image = rgb2gray(np.array(image))
    gray_image = (gray_image * 255).astype('uint8')
    lbp = local_binary_pattern(gray_image, P=24, R=3, method="uniform")  # 增加P值和R值
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 26), range=(0, 26))  # 相应地增加直方图的bins数
    return lbp_hist


def extract_hog_features(image):
    # 将PIL.Image对象转换为numpy数组，并转换为灰度
    gray_image = rgb2gray(np.array(image))
    # 确保图像尺寸一致，或调整HOG参数使特征长度一致
    resized_image = resize(gray_image, (128, 128))  # 例如，调整到128x128大小
    hog_features = hog(resized_image, orientations=8, pixels_per_cell=(16, 16),
                       cells_per_block=(1, 1), block_norm='L2-Hys', visualize=False, feature_vector=True)
    return hog_features

In [4]:
def extract_features(dataset, feature_extractor):
    X, y = [], []
    for i in range(len(dataset)):
        image, score = dataset[i]
        features = feature_extractor(image)
        # print(f"Features length from {feature_extractor.__name__}: {len(features)}")  # 打印特征长度
        X.append(features)
        y.append(score)
    return np.array(X, dtype=object), np.array(y)  # 使用dtype=object可以暂时解决问题，但最好是有统一的长度


In [5]:
feature_extractors = {
    'Color': extract_color_features,
    'LBP': extract_lbp_features,
    'HOG': extract_hog_features
}

In [6]:
labels_file = '../data/Wilma Bainbridge_10k US Adult Faces Database/Memorability Scores/memorability-scores.xlsx'
root_dir = '../data/Wilma Bainbridge_10k US Adult Faces Database/10k US Adult Faces Database/Face Images'
df = pd.read_excel(labels_file)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = FacesMemorabilityDataset(train_df, root_dir)
test_dataset = FacesMemorabilityDataset(test_df, root_dir)

In [7]:
results = {}
for feature_name, extractor in feature_extractors.items():
    # 创建特征和标签
    X_train, y_train = extract_features(train_dataset, extractor)
    X_test, y_test = extract_features(test_dataset, extractor)

    # 特征标准化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 训练SVM
    svm_regressor = SVR(kernel='rbf')
    svm_regressor.fit(X_train_scaled, y_train)

    # 预测
    y_pred = svm_regressor.predict(X_test_scaled)

    # 计算斯皮尔曼相关性
    correlation, _ = spearmanr(y_test, y_pred)
    results[feature_name] = correlation

for feature_name, correlation in results.items():
    print(f"Spearman Correlation for {feature_name}: {correlation:.2f}")

Spearman Correlation for Color: 0.17
Spearman Correlation for LBP: 0.19
Spearman Correlation for HOG: 0.48


In [ ]:
for feature_name, correlation in results.items():
    print(f"Spearman Correlation for {feature_name}: {correlation:.2f}")